# Recipes

This notebook is a collection of recipes;
useful data manipulations or plots commonly used by users.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

path = "osyrisdata/starformation"
data5 = osyris.Dataset(5, path=path).load()
data8 = osyris.Dataset(8, path=path).load()

## Difference between two snapshots: histograms

Here, we want to make a map of the difference in 2D histograms between two snapshots (outputs `5` and `8`).
Because we do not necessarily have the same number of cells at the same place,
we first have to make uniform 2D maps using the `histogram2d` function,
which we can then directly compare.

The `histogram2d` function actually returns an object that contains the raw data that was used to create the figure.
By using the `plot=False` argument, we can silence the figure generation, and use the data in a custom figure.

**Note:** For the comparison to make sense, both histograms have to use the same horizontal and vertical range.

In [ ]:
hist5 = osyris.histogram2d(data5["hydro"]["density"], data5["hydro"]["B_field"],
                           norm="log", loglog=True, plot=False)

hist8 = osyris.histogram2d(data8["hydro"]["density"], data8["hydro"]["B_field"],
                           norm="log", loglog=True, plot=False)

# Get x,y coordinates
x = hist5.x
y = hist5.y

# Difference in counts
counts5 = np.log10(hist5.layers[0]["data"])
counts8 = np.log10(hist8.layers[0]["data"])
diff = (counts5 - counts8) / counts8

# Create figure
fig, ax = plt.subplots()
cf = ax.contourf(x, y , diff, cmap='RdBu',
                 levels=np.linspace(-0.3,0.3,11))
ax.set_xscale('log')
ax.set_yscale('log')
cb = plt.colorbar(cf, ax=ax)
cb.ax.set_ylabel("Relative difference")

## Difference between two snapshots: spatial maps

Here, we want to make a map of the difference in density between two snapshots.
The `map` function also returns an object that contains the raw data that was used to create the figure.
By using the `plot=False` argument, we can silence the figure generation, and use the data in a custom figure.

**Note:** For this to make sense, the two outputs have to be centered around the same center point.

In [ ]:
# Find center
ind = np.argmax(data8["hydro"]["density"])
center = data8["amr"]["position"][ind]

dx = 2000 * osyris.units("au")
# Extract density slices by copying data into structures
plane5 = osyris.map({"data": data5["hydro"]["density"], "norm": "log"},
                    dx=dx, origin=center, direction="z", plot=False)

plane8 = osyris.map({"data": data8["hydro"]["density"], "norm": "log"},
                    dx=dx, origin=center, direction="z", plot=False)

# Get x,y coordinates
x = plane5.x
y = plane5.y

# Density difference
rho5 = np.log10(plane5.layers[0]["data"])
rho8 = np.log10(plane8.layers[0]["data"])
diff = (rho5 - rho8) / rho8

# Create figure
fig, ax = plt.subplots()
cf = ax.contourf(x, y , diff, cmap='RdBu',
                 levels=np.linspace(-0.12,0.12,31))
ax.set_aspect("equal")
cb = plt.colorbar(cf, ax=ax)
cb.ax.set_ylabel("Relative difference")

## Radial profile

This example shows how to make a 1D profile of the mean gas density as a function of radius.

In [ ]:
# Re-center cell coordinates according to origin
data8['amr']['pos_new'] = (data8['amr']['position'] - center).to("au")

# Create figure
fig, ax = plt.subplots()

# Make scatter plot as radial profile
step = 100
osyris.scatter(data8['amr']['pos_new'][::step],
               data8['hydro']['density'][::step],
               c='grey', edgecolors='None', loglog=True, ax=ax)

# Define range and number of bins
rmin = 1.5
rmax = 3.5
nr = 100

# Radial bin edges and centers
edges = np.linspace(rmin, rmax, nr+1)
midpoints = 0.5 * (edges[1:] + edges[:-1])

# Bin the data in radial bins
z0, _ = np.histogram(np.log10(data8['amr']['pos_new'].norm.values), bins=edges)
z1, _ = np.histogram(np.log10(data8['amr']['pos_new'].norm.values), bins=edges,
                     weights=data8['hydro']['density'].values)
rho_mean = z1/z0

#Overlay profile
ax.plot(10.0**midpoints, rho_mean, color='r', lw=3, label="Mean profile")
ax.legend()